In [4]:
import pandas as pd

import re
import string
import nltk

nltk.download("stopwords")
nltk.download("wordnet")
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import contractions

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hongong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/hongong/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
# Load dataset
df = pd.read_pickle("data/IMDB-Dataset.pkl", compression="bz2")
print(df.shape)
df.head()

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
# Remove duplicate rows
df = df.drop_duplicates()
print(df.shape)

(49582, 2)


In [24]:
stop = set(stopwords.words("english"))


# Expanding contractions
def expand_contractions(text):
    return contractions.fix(text)


# Function to clean data
def preprocess_text(text):
    wl = WordNetLemmatizer()
    soup = BeautifulSoup(text, "html.parser")  # Removing html tags
    text = soup.get_text()
    text = expand_contractions(
        text
    )  # Expanding chatwords and contracts clearing contractions

    #
    emoji_clean = re.compile(
        "["
        "\U0001F600-\U0001F64F"
        "\U0001F300-\U0001F5FF"
        "\U0001F680-\U0001F6FF"
        "\U0001F1E0-\U0001F1FF"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )

    text = emoji_clean.sub(r"", text)
    text = re.sub(r"\.(?=\S)", ". ", text)  # add space after full stop
    text = re.sub(r"http\S+", "", text)  # remove urls
    text = "".join(
        [word.lower() for word in text if word not in string.punctuation]
    )  # remove punctuation and make text lowercase
    text = " ".join(
        [
            wl.lemmatize(word)
            for word in text.split()
            if word not in stop and word.isalpha()
        ]
    )  # lemmatize

    return text

In [29]:
%%time
df["review_preprocessed"] = df["review"].apply(preprocess_text)

/var/folders/y2/hzjfncr92llg100m08nwxmhc0000gn/T/ipykernel_43110/606729018.py:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")  # Removing html tags


CPU times: user 22.6 s, sys: 211 ms, total: 22.8 s
Wall time: 23.1 s


In [30]:
df.head()

,review,sentiment,review_preprocessed
0,One of the other reviewers has mentioned that ...,positive,one reviewer mentioned watching oz episode hoo...
1,A wonderful little production. <br /><br />The...,positive,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,negative,basically family little boy jake think zombie ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter matteis love time money visually stunni...


In [31]:
df.to_pickle("data/preprocess_data.pkl", compression="bz2")